In [ ]:
from math import exp, log, sqrt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.optimize as opt

from linear_motion_sim import Simulator, fit_ks_and_kv, fit_ka, lognorm
from func_guesser import sign

In [ ]:
sim = Simulator(max_speed=20, max_accel=12, drag=0.8, noise=0.2)

In [ ]:
ks, kv = fit_ks_and_kv(sim)
ka = fit_ka(sim, ks, kv)

In [ ]:
ks *= lognorm(0.05)
kv *= lognorm(0.05)
ka *= lognorm(0.1)

In [ ]:
ac = sim.sim(lambda t, x, v, a: -ks + kv * v + ka * -2)
sim.plot_curves(ac)

In [ ]:
vtol = 0.1
xtol = 0.5
ttol = 0.1

In [ ]:
def ratelimit(x1, t1, x0, t0, v):
    dt = t1 - t0
    up = x0 + v * dt
    dn = x0 - v * dt
    return min(max(x1, dn), up)

In [ ]:
def smaller_mag(x, y):
    if abs(x) < abs(y):
        return x
    else:
        return y

In [ ]:
class Drive:
    def __init__(self,  
                 tgt_x,
                 tgt_v,
                 alimit, 
                 vlimit):
        self.tgt_x = tgt_x
        self.tgt_v = tgt_v
        self.alimit = alimit
        self.vlimit = vlimit
        self.done = False
        self.prev_t = 0
        self.prev_p = 0


    def drive_power(self, t, x, v, a):
        if self.done:
            return 0.05 * np.random.normal()

        if abs(x - self.tgt_x) < xtol and abs(v - self.tgt_v) < vtol:
            self.done = True
            return 0
        
        dist = self.tgt_x - x
        s = sign(dist)
        v0 = sqrt(2 * self.alimit * abs(dist))
        v1 = s * min(v0, self.vlimit)

        dt = t - self.prev_t
        
        next_a = smaller_mag(
            (v1 - v) / dt / 1.2, 
            self.alimit * sign(v1 - v))

        p = ks * s + kv * v + ka * next_a

        self.prev_t = t

        return p

In [ ]:
d = Drive(tgt_x=+60, tgt_v=0, alimit=8, vlimit=12)
ac = sim.sim(d.drive_power, max_t=15)
sim.plot_curves(ac, tgt_x=d.tgt_x)